---
<span style="color:orange;font-size:30px;">PRELIMINAR BEGIN</span>
---


In [1]:
%%configure
{ "conf": {
  "spark.jars":"hdfs:///apps/hudi/lib/hudi-spark-bundle.jar,hdfs:///apps/hudi/lib/spark-avro.jar",
  "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
  "spark.sql.hive.convertMetastoreParquet":"false"
}}

In [2]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1689180971920_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
##CALL SOME LIBRARIES IF WHERE NECESARY
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col, to_timestamp, monotonically_increasing_id, to_date, when
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, array, ArrayType, DateType, DecimalType, TimestampType, DoubleType
from pyspark.sql.functions import col, to_timestamp, monotonically_increasing_id, to_date, when,from_utc_timestamp
from pyspark.sql.functions import input_file_name, regexp_extract, isnan, isnull
from pyspark.sql.types import *
from datetime import datetime, timedelta
import time

from pyspark.sql import DataFrame
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.session import SparkSession
from datetime import datetime
import hashlib

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

---
<span style="color:green;font-size:30px;">READING THE DATA LIST FILES</span>
---


In [4]:
#i got the list of files to read there were in a csv
s3_path='s3://mloaiza.test/files_all.csv'
df_list_files = spark.read.csv(s3_path, header=True,sep=';')
df_list_files.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- RUTA: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- size: string (nullable = true)

In [5]:
###convert the RUTA "PATH" filed in a list of files
list_files = df_list_files.select('RUTA').distinct().rdd.flatMap(
    lambda x: x).collect() 


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
#we read the data from a list of files
df = spark.read.format("csv").option("inferSchema", "true").option("header", "true").option("sep",",").load(list_files)
df.printSchema()
df.persist()
df.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _c0: integer (nullable = true)
 |-- patente: string (nullable = true)
 |-- mensaje_aceptacion: string (nullable = true)
 |-- fecha_consulta: string (nullable = true)

In [33]:
df_total=df
df_total.persist()
df_total.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-------+--------------------+--------------------+
|_c0|patente|  mensaje_aceptacion|      fecha_consulta|
+---+-------+--------------------+--------------------+
|  0| TP9749|no está sujeta a ...|10 de Julio de 20...|
|  0| SKXJ32|no está sujeta a ...|10 de Julio de 20...|
|  0| SCFG49|no está sujeta a ...|10 de Julio de 20...|
|  0| CHXR24|no está sujeta a ...|10 de Julio de 20...|
|  0| RBPW66|no está sujeta a ...|10 de Julio de 20...|
|  0| CPKX86|no está sujeta a ...|10 de Julio de 20...|
|  0| LWVB12|no está sujeta a ...|10 de Julio de 20...|
|  0| DCPW43|no está sujeta a ...|10 de Julio de 20...|
|  0| LFZS28|no está sujeta a ...|10 de Julio de 20...|
|  0| LFDZ38|no está sujeta a ...|10 de Julio de 20...|
|  0| KBVZ63|no está sujeta a ...|10 de Julio de 20...|
|  0| DTDV62|no está sujeta a ...|10 de Julio de 20...|
|  0| JSHB80|no está sujeta a ...|10 de Julio de 20...|
|  0| JRFP50|no está sujeta a ...|10 de Julio de 20...|
|  0| JJYY89|no está sujeta a ...|10 de Julio de

In [34]:
df_total=df_total.distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
df_total.persist()
df_total.show()
df_total.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-------+--------------------+--------------------+
|_c0|patente|  mensaje_aceptacion|      fecha_consulta|
+---+-------+--------------------+--------------------+
|  0| GHTT42|no está sujeta a ...|9 de Julio de 202...|
|  0| DLBG73|no está sujeta a ...|9 de Julio de 202...|
|  0| GTPY41|no está sujeta a ...|9 de Julio de 202...|
|  0| UT5916|no está sujeta a ...|9 de Julio de 202...|
|  0| YP7538|no está sujeta a ...|9 de Julio de 202...|
|  0| VL9278|no está sujeta a ...|9 de Julio de 202...|
|  0| YE4691|no está sujeta a ...|10 de Julio de 20...|
|  0| KTXP38|no está sujeta a ...|10 de Julio de 20...|
|  0| HCLH11|no está sujeta a ...|10 de Julio de 20...|
|  0| JDTV43|no está sujeta a ...|9 de Julio de 202...|
|  0| JCLX94|no está sujeta a ...|9 de Julio de 202...|
|  0| LPKL65|no está sujeta a ...|10 de Julio de 20...|
|  0| UF2700|no está sujeta a ...|10 de Julio de 20...|
|  0| CJWB95|no está sujeta a ...|10 de Julio de 20...|
|  0| LLJV93|no está sujeta a ...|10 de Julio de

---
<span style="color:red;">BEGIN ETL</span>
---

In [36]:
df_total = df_total.select('patente','mensaje_aceptacion','fecha_consulta')

def get_fecha_consulta(text):
    month_dict = {
        "enero": 1, "febrero": 2, "marzo": 3, "abril": 4, "mayo": 5, "junio": 6, "julio": 7, "agosto": 8, "septiembre": 9, "octubre": 10,
        "noviembre": 11, "diciembre": 12
    }
    new_text = text.replace("de", "").lower()
    values = new_text.split(" ")
    values = [i for i in values if i != '']
    day = values[0]
    month = month_dict[values[1]]
    year = values[2]
    date_consulta = datetime.date(int(year), int(month), int(day))
    return date_consulta

df_total.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- patente: string (nullable = true)
 |-- mensaje_aceptacion: string (nullable = true)
 |-- fecha_consulta: string (nullable = true)

In [40]:
from pyspark.sql.functions import udf
from datetime import datetime, timedelta
import datetime

year='2023'
month='7'
day='10'

df=df_total
udf_get_fecha_consulta = udf(get_fecha_consulta, DateType())
# Assuming your DataFrame is named "df" and the column is "fecha_consulta"
df = df.withColumn("fecha_consulta2", udf_get_fecha_consulta(df.fecha_consulta))
df = df.select('patente','mensaje_aceptacion','fecha_consulta2')
df = df.withColumnRenamed("fecha_consulta2", "fecha_consulta")
df = df.withColumn("fecha", lit(datetime.date(int(year), int(month), int(day))))
df = df.withColumn("year", lit(int(year)))
df = df.withColumn("month", lit(int(month)))
df = df.withColumn("day", lit(int(day)))

df = df.select('patente','mensaje_aceptacion','fecha_consulta', 'fecha', 'year', 'month', 'day')
df.persist()
df.printSchema()
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- patente: string (nullable = true)
 |-- mensaje_aceptacion: string (nullable = true)
 |-- fecha_consulta: date (nullable = true)
 |-- fecha: date (nullable = false)
 |-- year: integer (nullable = false)
 |-- month: integer (nullable = false)
 |-- day: integer (nullable = false)

+-------+--------------------+--------------+----------+----+-----+---+
|patente|  mensaje_aceptacion|fecha_consulta|     fecha|year|month|day|
+-------+--------------------+--------------+----------+----+-----+---+
| GHTT42|no está sujeta a ...|    2023-07-09|2023-07-10|2023|    7| 10|
| DLBG73|no está sujeta a ...|    2023-07-09|2023-07-10|2023|    7| 10|
| GTPY41|no está sujeta a ...|    2023-07-09|2023-07-10|2023|    7| 10|
| UT5916|no está sujeta a ...|    2023-07-09|2023-07-10|2023|    7| 10|
| YP7538|no está sujeta a ...|    2023-07-09|2023-07-10|2023|    7| 10|
| VL9278|no está sujeta a ...|    2023-07-09|2023-07-10|2023|    7| 10|
| YE4691|no está sujeta a ...|    2023-07-10|2023-07-10|2023|  

In [41]:
path='s3://mloaiza.test/processced_2023_7_10/'
df=df.repartition(100).coalesce(50)
df.persist()
df.write.mode('overwrite').parquet(path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

---
<span style="font-size:30px;color:green">OLD GARBAGE</span> <i class="fa fa-recycle"></i>
---
